# Import Libraries

In [45]:
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report , confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Activation, Dense, Reshape, Input
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.models import Model , clone_model
from keras.losses import BinaryCrossentropy
from keras.backend import clear_session
from keras.initializers import Constant
from nltk.tokenize import word_tokenize
from threading import current_thread
from keras.utils import plot_model
from sklearn.utils import shuffle
from keras.optimizers import Adam
from keras.utils import Sequence
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from functools import partial
from numpy import asarray
import concurrent.futures
from keras import layers
from keras import Model
from numpy import save
from numpy import load
import esprima as esp
from time import time
import pandas as pd
import numpy as np
import logging
import urllib
import random
import pickle
import pydot
import torch
import math
import html
import nltk
import os
import re
import gc
np.random.seed(123)
plt.style.use('ggplot')

In [46]:
# with open("G:/JSContanaDataSet/programs_training.txt", 'r', encoding='utf-8') as file:
#     path_scripts = file.readlines()
# df_0 = pd.DataFrame(columns=["ScriptPath","Label"])
# scripts = []
# Label = []
# for path_script in range(len(path_scripts)-1):
#     try:
#         s_p = "G:/JSContanaDataSet/"+path_scripts[path_script][:-1]
#         with open(s_p) as js_file:
#             script = js_file.read()
#         scripts.append(path_scripts[path_script][:-1])
#         Label.append(0)
#     except:
#         pass
# df_0["ScriptPath"] = scripts
# df_0["Label"] = Label
# df_0 = df_0.sample(40000)
# df_0.to_csv("G:\JSContanaDataSet\df_0.csv", index=False)

In [47]:
# import os
# path = "G:\JSContanaDataSet\javascript-malware-collection-master"
# os.chdir(path)
# ListOfD = os.listdir()
# ListOfD

In [48]:
# de = []
# df_1 = pd.DataFrame(columns=["ScriptPath","Label"])
# for directory in ListOfD:
#     if not ("." in directory):
#         mypath = os.path.join(path,directory)
#         mypath1 = os.path.join("javascript-malware-collection-master",directory)
#         os.chdir(mypath)
#         for d in os.listdir():
#             my_path = os.path.join(mypath,d)
#             my_path1 = os.path.join(mypath1,d)
#             if not ("." in d):
#                 os.chdir(my_path)
#                 for j in os.listdir():
#                     print(os.path.join(my_path1,j))
#                     print(os.path.join(my_path,j))
#                     if (".js") in j:
#                         de.append(os.path.join(my_path1,j))
# df_1["ScriptPath"] = scripts
# df_1["Label"] = Label
# df_1.to_csv("G:\JSContanaDataSet\df_1.csv", index=False)

# Preprocessing_Files

In [49]:
def Decode_Stage(script):
    script = urllib.parse.unquote(script)
    script = html.unescape(script)
    return script

In [50]:
def Generalization_Stage(x):
    x = x.lower()
    x, _ = re.subn(r'\d+', "0", x)
    x, _ = re.subn(r'(http|https)://[a-zA-Z0-9\.@/!&#\?]+',"http://u", x)
    return x

In [51]:
def Segmentation_Stage(x):
    seg = r'''(?x)[\w\.]+?\(|\)|"\w+?"|'\w+?'|http://\w|</\w+>|<\w+>|<\w+|\w+=|>|[\w\.]+/'''
    token = nltk.regexp_tokenize(x, seg)
    return token

In [52]:
def remove_comments(script):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        if match.group(2) is not None:
            return ""
        else:
            return match.group(1)
    return regex.sub(_replacer, script)

In [53]:
def lowering_remove_urls_ips(script):
    """
    _ INPUT IS JS FILE 
    _ OUTPUT IS CLEAN JS FILE
    """
    script = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', script)
    script = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)', '0', script)
    script = remove_comments(script)
    script = script.lower()
    return script

In [54]:
# convert java script code to the abstract syntax tree
# then extract the sequences of syntax units with detailed information
def sequence_of_syntax_units(scripts):
    try:
        """
        sctipts : list or single javascript code 
        return sequence of syntax units
        each syntax unit correspond to line in an abstract syntax tree
        """
        if type(scripts) is list:
            # esp.parseScript(script) returns abstract syntax tree of each js scripts
            return [re.sub('\s+', ' ', ''.join(str(esp.parseScript(script)).split('\n'))) for script in scripts]
        elif type(scripts) is str:
            return re.sub('\s+', ' ', ''.join(str(esp.parseScript(scripts)).split('\n')))
        else:
            raise ValueError('The type of scripts parameter must be {list or string}')
    except:
        pass
    return None

In [55]:
def read__script(script_paths,parent_path):
    try :
        with open(os.path.join(parent_path,script_paths), 'r') as js_file:
            script = js_file.read()
            script = Generalization_Stage(script)
            script = lowering_remove_urls_ips(script)
            script = sequence_of_syntax_units(script)
        return script
    except:
        print(script_paths)
    return None

In [74]:
df_0 = pd.read_csv("G:\JSContanaDataSet\df_0.csv")
print(df_0.shape)
df_0.head()

(40000, 2)


,ScriptPath,Label
0,data\phatly27\light-responsive-js\node_modules...,0
1,data\AshKyd\geojson-regions\example\leaflet.la...,0
2,data\mozilla\mozmill\mutt\mutt\tests\js\metro\...,0
3,data\ehynds\grunt-image-embed\test\test_image_...,0
4,data\stackgl\webgl-workshop\exercises\shader-a...,0


In [75]:
df_1 = pd.read_csv("G:\JSContanaDataSet\df_1.csv")
print(df_1.shape)
df_1.head()

(39425, 2)


,ScriptPath,Label
0,javascript-malware-collection-master\1936\1936...,1
1,javascript-malware-collection-master\1936\1936...,1
2,javascript-malware-collection-master\1936\1936...,1
3,javascript-malware-collection-master\1936\1936...,1
4,javascript-malware-collection-master\1936\1936...,1


In [76]:
from sklearn.utils import shuffle
# df_0 = df_0[-5000:-3000]
df_1 = df_1[10000:17000]
frames = df_1 # [df_1, df_0]
# frames = pd.concat(frames)
df = shuffle(frames)
df.to_csv("G:\JSContanaDataSet\my_df2.csv", index=False)

In [77]:
my_df = pd.read_csv("G:\JSContanaDataSet\my_df2.csv")
print(my_df.shape)
my_df.head()

(7000, 2)


,ScriptPath,Label
0,javascript-malware-collection-master\2016\2016...,1
1,javascript-malware-collection-master\2016\2016...,1
2,javascript-malware-collection-master\2016\2016...,1
3,javascript-malware-collection-master\2016\2016...,1
4,javascript-malware-collection-master\2016\2016...,1


In [78]:
def get_js_File_after_preprocessing(my_df):
    list_X_train = []
    list_y_train = []
    for i,df_iterator in tqdm(my_df.iterrows()):
        X_train , y_train = read__script(df_iterator[my_df.columns[-2]],"G:\JSContanaDataSet"),df_iterator[my_df.columns[-1]]
        if not X_train is None:
            list_X_train.append(X_train)
            list_y_train.append(y_train)
    return list_X_train,list_y_train

In [ ]:
list_X_train,list_y_train = get_js_File_after_preprocessing(my_df)

In [ ]:
print(len(list_X_train))
print(len(list_y_train))
my_df = pd.DataFrame()
my_df['ScriptPath'] = list_X_train
my_df['Label'] = list_y_train
my_df.to_csv("G:\JSContanaDataSet\my_df2.csv", index=False)
my_df = pd.read_csv("G:\JSContanaDataSet\my_df2.csv")
print(my_df.shape)
my_df.head()

In [ ]:
print(my_df[my_df["Label"]==0].shape)
print(my_df[my_df["Label"]==1].shape)

In [ ]:
my_df__ = pd.read_csv("G:\JSContanaDataSet\my_df2.csv")
print(my_df__.shape)
print(my_df__[my_df__["Label"]==0].shape)
print(my_df__[my_df__["Label"]==1].shape)
my_df__.head()